### Goal
Run flows using sdk and allowing for external configuration of RAG parameters. 

### Check Connections

In [2]:
import promptflow

pf = promptflow.PFClient()

# List all the available connections
for c in pf.connections.list():
    print(c.name + " (" + c.type + ")")

azureopenai_local (AzureOpenAI)
aoai_connection (AzureOpenAI)
default_connection (CognitiveSearch)
cosmodb_transcript (Custom)
cosmodb_connection (Custom)
acs_connection (CognitiveSearch)


### Test the flow

In [14]:
output = pf.flows.test(
    "rag-copilot/flow_config.dag.yaml",
    inputs={
        "chat_history": [],
        "query": "What is the growth rate of Azure ML revenue in FY23Q1?",
        "TOP_K":5
    },
)

print(output)

{'fetched_docs': '{"retrieved_documents": [{"[doc0]": {"ticker": "MSFT", "quarter": "1", "year": "23", "content": "hiring resulting in mid-to- high-single-digits revenue growth or low to mid- teens growth in constant currency. And in Dynamics, we expect revenue growth in the low-double-digits or the low 20s in constant currency driven by continued share gains in Dynamics 365. For Intelligent Cloud we expect revenue to grow between 22 and 24 percent in constant currency or $21.25 to $21.55 billion US dollars. Revenue will continue to be driven by Azure which, as a reminder, can have quarterly variability primarily from our per-user business and from in- period revenue recognition depending on the mix of contracts. "}}, {"[doc1]": {"ticker": "MSFT", "quarter": "1", "year": "23", "content": "contracts. As a reminder, the growing mix of larger long-term Azure contracts, which are more unpredictable in their timing, always drives increased quarterly volatility in our bookings growth rate. M

### Run the flow with the benchmark data file

In [15]:
flow_path = "rag-copilot/flow_config.dag.yaml"
data_path = "./rag-copilot/evalset.csv"

column_mapping = {
  "chat_history" :"${data.chat_history}",
  "query": "${data.question}",
  "TOP_K":3,
}

run_topk3 = pf.run(flow = flow_path, data = data_path, column_mapping=column_mapping)
pf.stream(run_topk3)
print(run_topk3)

2024-01-10 12:16:20 -0800   21260 execution.bulk     INFO     Process 21260 started.
2024-01-10 12:16:20 -0800    1396 execution.bulk     INFO     Process 1396 started.
2024-01-10 12:16:20 -0800   34240 execution.bulk     INFO     Process 34240 started.
2024-01-10 12:16:20 -0800   34360 execution.bulk     INFO     Process 34360 started.
2024-01-10 12:16:20 -0800   15740 execution.bulk     INFO     Process 15740 started.
2024-01-10 12:16:20 -0800    3444 execution.bulk     INFO     Process 3444 started.
2024-01-10 12:16:20 -0800   11320 execution.bulk     INFO     Process 11320 started.
2024-01-10 12:16:20 -0800   29272 execution.bulk     INFO     Process 29272 started.
======= Run Summary =======

Run name: "rag_copilot_variant_0_20240110_121614_534008"
Run status: "Completed"
Start time: "2024-01-10 12:16:14.534008"
Duration: "0:00:14.595503"
Output path: "C:\Users\hokhadiv\.promptflow\.runs\rag_copilot_variant_0_20240110_121614_534008"

name: rag_copilot_variant_0_20240110_121614_534

In [6]:
pf.get_details(run_topk3)

,inputs.chat_history,inputs.query,inputs.TOP_K,inputs.line_number,inputs.config,outputs.fetched_docs,outputs.filter,outputs.reply
0,[],What is the quarterly revenue of Microsoft Clo...,3,0,{'TOP_K': 5},"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'Year': '23', 'Quarter': '1'}","According to [doc0], the Microsoft Cloud excee..."
1,[],How many Azure Arc customers does Microsoft cu...,3,1,{'TOP_K': 5},"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'Year': '23', 'Quarter': '1'}","According to the retrieved documents, Microsof..."
2,[],Which companies run SAP on Azure in FY23Q1?,3,2,{'TOP_K': 5},"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'Year': '23', 'Quarter': '1'}","According to the retrieved documents, companie..."
3,[],What is the annual recurring revenue of GitHub...,3,3,{'TOP_K': 5},"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'Year': '23', 'Quarter': '1'}","According to the retrieved documents, GitHub h..."
4,[],How many monthly active users does Power BI ha...,3,4,{'TOP_K': 5},"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'Year': '23', 'Quarter': '1'}","According to the retrieved documents, Power BI..."
5,[],What is the quarterly revenue of Microsoft Clo...,3,5,{'TOP_K': 5},"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'Year': '23', 'Quarter': '2'}","According to the retrieved documents, the Micr..."
6,[],How many Azure Arc customers does Microsoft cu...,3,6,{'TOP_K': 5},"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'Year': '23', 'Quarter': '2'}","According to the retrieved document, Microsoft..."
7,[],What are some examples of companies using Azur...,3,7,{'TOP_K': 5},"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'Year': '23', 'Quarter': '2'}","According to the retrieved documents, in FY23Q..."
8,[],How many developers are using GitHub Copilot a...,3,8,{'TOP_K': 5},"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'Year': '23', 'Quarter': '2'}","According to the retrieved documents, GitHub i..."
9,[],What is the growth rate of Azure Machine Learn...,3,9,{'TOP_K': 5},"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'Year': '23', 'Quarter': '1'}",The retrieved documents do not provide any inf...


## Evaluate 

In [9]:
eval_flow_path = "evaluator/eval_multiple_score/"
data_path = "./rag-copilot/evalset.csv"

eval_run_topk3 = pf.run(
    flow = eval_flow_path,
    run = run_topk3,
    data = data_path,
    column_mapping = {
      "question": "${data.question}",
      "answer": "${data.answer}",
      "reply": "${run.outputs.reply}",
      "context": "${run.outputs.fetched_docs}",
    },
    display_name = "eval_multiple_score_topk3"
)
pf.stream(eval_run_topk3)
print(eval_run_topk3)


2024-01-10 12:06:34 -0800   30468 execution.bulk     INFO     Process 30468 started.
2024-01-10 12:06:34 -0800   22696 execution.bulk     INFO     Process 22696 started.
2024-01-10 12:06:34 -0800   12428 execution.bulk     INFO     Process 12428 started.
2024-01-10 12:06:34 -0800   11336 execution.bulk     INFO     Process 11336 started.
2024-01-10 12:06:34 -0800   26784 execution.bulk     INFO     Process 26784 started.
2024-01-10 12:06:34 -0800   32300 execution.bulk     INFO     Process 32300 started.
2024-01-10 12:06:34 -0800   30872 execution.bulk     INFO     Process 30872 started.
2024-01-10 12:06:34 -0800   25884 execution.bulk     INFO     Process 25884 started.
2024-01-10 12:06:34 -0800   30556 execution.bulk     INFO     Process 30556 started.
======= Run Summary =======

Run name: "eval_multiple_score_variant_0_20240110_120631_561080"
Run status: "Completed"
Start time: "2024-01-10 12:06:31.561080"
Duration: "0:00:08.431762"
Output path: "C:\Users\hokhadiv\.promptflow\.runs

In [11]:
pf.get_details(eval_run_topk3)

,inputs.question,inputs.answer,inputs.reply,inputs.context,inputs.line_number,outputs.groundedness,outputs.llm_similarity
0,What is the quarterly revenue of Microsoft Clo...,"$25 billion in quarterly revenue, up 24 percen...","According to [doc0], the Microsoft Cloud excee...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...",0,8,10
1,How many Azure Arc customers does Microsoft cu...,"More than 8,500 Arc customers, more than doubl...","According to the retrieved documents, Microsof...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...",1,10,10
2,Which companies run SAP on Azure in FY23Q1?,"Companies like Chobani, Munich RE, Sodexo, Vol...","According to the retrieved documents, companie...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...",2,8,10
3,What is the annual recurring revenue of GitHub...,GitHub is now at $1 billion in annual recurrin...,"According to the retrieved documents, GitHub h...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...",3,10,10
4,How many monthly active users does Power BI ha...,Power BI is the market leader in business inte...,"According to the retrieved documents, Power BI...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...",4,2,0
5,What is the quarterly revenue of Microsoft Clo...,"$27 billion in quarterly revenue, up 22 percen...","According to the retrieved documents, the Micr...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...",5,10,10
6,How many Azure Arc customers does Microsoft cu...,"More than 12,000 Arc customers, double the num...","According to the retrieved document, Microsoft...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...",6,10,10
7,What are some examples of companies using Azur...,"Over 200 customers, including KPMG and Al Jaze...","According to the retrieved documents, in FY23Q...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...",7,10,8
8,How many developers are using GitHub Copilot a...,More than one million people have used GitHub ...,"According to the retrieved documents, GitHub i...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...",8,8,8
9,What is the growth rate of Azure Machine Learn...,Azure ML revenue alone has increased more than...,The retrieved documents do not provide any inf...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...",9,1,0


In [12]:
pf.get_metrics(eval_run_topk3)

{'llm_similarity': 7.6, 'llm_groundedness': 7.7}

In [13]:
pf.visualize(eval_run_topk3)

The HTML file is generated at 'C:\\Users\\hokhadiv\\AppData\\Local\\Temp\\pf-visualize-detail-uz15h_pt.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
